# 라이브러리 설치

In [ ]:
!pip install torch==1.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 1.8 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0+cu121
    Uninstalling torch-2.1.0+cu121:
      Successfully uninstalled torch-2.1.0+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 1.11.0 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 1.11.0 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 1.11.0 which is incompatible.
torchvision 0.16.0+cu121 requires torch==2.1.0, but you have torch 1.11.0 which is incompatible.


In [ ]:

!pip install transformers==4.30.2

In [ ]:

!pip install pandas==2.0.3

In [ ]:

!pip install numpy==1.24.3

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.optim import Adam
import torch.nn.functional as F

# 데이터 전처리

In [ ]:
head= '/content/'
train_x = np.load(head+'train_x.npy') # 파일 불러와서 확인
test_x = np.load(head+'test_x.npy') # 파일 불러와서 확인

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv")

In [ ]:
df_need = df[df.label != 0] # label이 0인 값 제거

In [ ]:
df_shuffled = df_need.sample(frac=1).reset_index(drop=True)

In [ ]:
df_shuffled = df_shuffled.loc[:, ['Q', 'label']]

In [ ]:
df_shuffled

,Q,label
0,두번째 이별,1
1,힘드네. 정말.이런건 줄 몰랐어,1
2,나 어장인듯,2
3,오래 연애하고 싶어.,2
4,나만 힘든건지,1
...,...,...
6528,쌈은 뭐야,2
6529,텅빔,1
6530,내가 싫은 건 너도 싫어해줘서 고마워,2
6531,짝녀에게 다가가려면 짝녀의 친구에게 먼저 다가가는게 나을까?,2


In [ ]:
train= df_shuffled[:4000]
test = df_shuffled[4000:5000]

# pytorch 이용한 모델 학습 시도

In [ ]:
class NsmcDataset(Dataset):
    ''' Naver Sentiment Movie Corpus Dataset '''
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, 0]
        label = self.df.iloc[idx, 1]
        return text, label

In [ ]:
nsmc_train_dataset = NsmcDataset(train)

In [ ]:
nsmc_train_dataset

In [ ]:
model_name = "sangrimlee/bert-base-multilingual-cased-nsmc"

In [ ]:
device_name = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.to(device)

pytorch_model.bin:   0%|          | 0.00/712M [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
optimizer = Adam(model.parameters(), lr=1e-5)

itr = 1
p_itr = 500
epochs = 1
total_loss = 0
total_len = 0
total_correct = 0

In [ ]:
train_loader = DataLoader(nsmc_train_dataset, batch_size=2, shuffle=True)

In [ ]:
model.train()
for epoch in range(epochs):

    for text, label in train_loader:
        optimizer.zero_grad()

        encoded = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")
        encoded, label = encoded.to(device), label.to(device)
        outputs = model(**encoded, labels=label)

        loss = outputs.loss
        logits = outputs.logits

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(label)
        total_correct += correct.sum().item()
        total_len += len(label)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr+=1

IndexError: Target 2 is out of bounds.

---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
<ipython-input-42-6392abe94f86> in <cell line: 2>()
      7         encoded = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")
      8         encoded, label = encoded.to(device), label.to(device)
----> 9         outputs = model(**encoded, labels=label)
     10 
     11         loss = outputs.loss

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py in _call_impl(self, *input, **kwargs)
   1108         if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks or _global_backward_hooks
   1109                 or _global_forward_hooks or _global_forward_pre_hooks):
-> 1110             return forward_call(*input, **kwargs)
   1111         # Do not call functions when jit is used
   1112         full_backward_hooks, non_full_backward_hooks = [], []

/usr/local/lib/python3.10/dist-packages/transformers/models/bert/modeling_bert.py in forward(self, input_ids, attention_mask, token_type_ids, position_ids, head_mask, inputs_embeds, labels, output_attentions, output_hidden_states, return_dict)
   1595             elif self.config.problem_type == "single_label_classification":
   1596                 loss_fct = CrossEntropyLoss()
-> 1597                 loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
   1598             elif self.config.problem_type == "multi_label_classification":
   1599                 loss_fct = BCEWithLogitsLoss()

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py in _call_impl(self, *input, **kwargs)
   1108         if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks or _global_backward_hooks
...
-> 2996     return torch._C._nn.cross_entropy_loss(input, target, weight, _Reduction.get_enum(reduction), ignore_index, label_smoothing)
   2997 
   2998 

IndexError: Target 2 is out of bounds.
Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...